In [2]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow import keras
os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5 # maximun alloc gpu50% of MEM
# config.gpu_options.allow_growth = True #allocate dynamically
# sess = tf.Session(config = config)

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import pytorch_lightning as pl
import torch

import os
import numpy as np
from PIL import Image


2024-12-10 19:36:54.197163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

BACKBONE = 'resnet34'
preprocess_input = get_preprocessing_fn(BACKBONE)

# load images and convert them to numpy arrays
real_images_dir = 'data/imagery/'
mask_images_dir = 'data/masks/'

x_train = []
y_train = []
x_val = []
y_val = []

for item in os.listdir(real_images_dir):
    if item.endswith('.png') and not item.startswith('test'):
        real_img = Image.open(real_images_dir + item)
        mask_img = Image.open(mask_images_dir + item)
        # Divide the image into 256x256 patches
        real_img_array = np.array(real_img)
        mask_img_array = np.array(mask_img)
        tiles_real = [real_img_array[x:x+72,y:y+72] for x in range(0,real_img_array.shape[0],72) for y in range(0,real_img_array.shape[1],72)]
        tiles_mask = [mask_img_array[x:x+72,y:y+72] for x in range(0,mask_img_array.shape[0],72) for y in range(0,mask_img_array.shape[1],72)]
        x_train.extend(tiles_real)
        y_train.extend(tiles_mask)

x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = y_train[:,:,:,:3]

In [4]:
class Model(pl.LightningModule):
    def __init__(self):
        super(Model, self).__init__()
        self.model = smp.Unet(BACKBONE, encoder_weights='imagenet')
        self.loss = smp.losses.DiceLoss()
        self.metrics = [smp.metrics.IoU(threshold=0.5)]

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [5]:
from segmentation_models_pytorch.losses import soft_bce
from segmentation_models_pytorch.metrics import iou_score
from torch.utils.data import DataLoader

print(x_train.shape)
print(y_train.shape)
x_train = x_train[:1000].astype('float32')
y_train = y_train[:1000].astype('float32')     

# preprocess input
x_train = preprocess_input(x_train)
# x_val = preprocess_input(x_val)

train_dataset = DataLoader(list(zip(x_train, y_train)), batch_size=10, shuffle=True)

# define model

# model = smp.Unet("resnet34", encoder_weights="imagenet", classes=3, activation=None)
model = Model()
trainer = pl.Trainer(max_epochs=100)
trainer.fit(model, train_dataset)

# image, mask = dataset[0]

(46464, 72, 72, 3)
(46464, 72, 72, 3)


TypeError: __init__() missing 1 required positional argument: 'mode'